Legacy functions for RDBS interaction

In [ ]:
import pymysql
endpoint = 'ta10projectinstance1.c0m59tujlbqm.ap-southeast-2.rds.amazonaws.com'
username = 'admin'
password = 'fit5120ta10'
database_name = 'fit5120_i3_schema'
client_flag = 'CLIENT.MULTI_STATEMENTS'

connection = pymysql.connect(host = endpoint, user = username, passwd = password, db = database_name)

# Function used to clear all entries in a single table considering no indexes or Foreign keys are present
def truncate_handler(table):
    cursor = connection.cursor()
    cursor.execute("""TRUNCATE {}""".format(table))
    connection.commit()
    return 'table {} has been truncated'.format(table)

def country_region_handler(values):
    query = """INSERT INTO country_region (country_region_id, country_code, region_name) VALUES (%s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def country_handler(values):
    query = """INSERT INTO country (country_name, country_code) VALUES (%s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def symptom_handler(values):
    query = """INSERT INTO symptom (symp_id, symp_name) VALUES (%s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def case_details_handler(values):
    query = """INSERT INTO case_details (case_id, symp_id) VALUES (%s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def case_handler(values):
    query = """INSERT INTO `case` (case_id, case_status, case_date, case_gender, country_region_id, case_death_date) VALUES (%s, %s, %s, %s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def prediction_handler(values):
    query = """INSERT INTO predictions (country_region_id, pred_date, pred_count) VALUES (%s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'predictions table has been updated'

def query(query):
    sql_query = """{}""".format(query)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    return cursor.fetchall()

Functions for data wrangling and extraction

In [ ]:
import pandas as pd
import datetime as dt
# Function used to load new datasource
def load_data():
    # Loads the data
    df = pd.read_csv('https://raw.githubusercontent.com/owid/monkeypox/main/owid-monkeypox-data.csv')
    # converts the date column to a datetime type
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.date
    # Drops the rows related to the global count
    df = df.loc[df['iso_code'] != 'OWID_WRL']
    return df

# Function used to check if a death has occured to populate case_death_date column
def death_date_check(row):
    if row['new_deaths'] == 0:
        return None
    else:
        return row['date']
# Function used to generate the country_region_id as the data is country level
def country_region_id_gen(row):
    c_r_id = row['iso_code'] + "_0"
    return c_r_id
# Function used to generate the case_id from the raw data
def case_id_gen(df):
    owid_entries = query("SELECT count(case_id) FROM `case` WHERE case_id LIKE 'OWID%'")[0][0]
    new_entries = df.shape[0]
    c_id = ['OWID_' + str(owid_entries + x) for x in range(0,new_entries)]
    return c_id

# Function used to extract the latest cases from the new dataset which have not been inserted into the rdbs yet
def new_cases(df, last_entry):
    # Selects all cases that have occured from a given date on wards
    df = df.loc[df['date'] > last_entry]
    # drops all columns which are not required to reduce computational loading
    df = df[['location', 'iso_code', 'date', 'new_deaths']]
    # Creates a new column for case_death_date
    df['case_death_date'] = df.apply(death_date_check, axis = 1)
    # Creates a new column for case_gender
    df['case_gender'] = None
    # Creates the country_region_id
    df['country_region_id'] = df.apply(country_region_id_gen, axis = 1)
    # Creates a case_id and sorts the column to be ordered by date than country_region_id
    df = df.sort_values(['date', 'country_region_id'], ascending=False)
    df['case_id'] = case_id_gen(df)
    # Populates the status column
    df['case_status'] = 'confirmed'
    # Renames all columns to the appropriate column names for importing
    df = df.rename({'date': 'case_date'}, axis = 1)
    # Selects a subset of the dataframe after all columns have been generated for the case table
    df = df[['case_id', 'case_status', 'case_date', 'case_gender', 'country_region_id', 'case_death_date']]
    #df = df[['case_status', 'case_date', 'case_gender', 'country_region_id', 'case_death_date']]
    return df

# Function used to find the new rows to be uploaded
def update_case(df):
    owid_entries = query('SELECT count(case_id) FROM `case` WHERE case_id LIKE "OWID%"')[0][0]
    # Calculates how many rows are missing or need to be added into the rdbs
    extract = df.shape[0] - owid_entries
    # Selects all entries besides the last entries equal to owid_entries
    df_select = df.iloc[:extract,]
    return df_select

Main function (to be run on a daily basis to keep rdbs up to date)

In [ ]:
def main():
    # Loads the new data
    mpx_data = load_data()
    # loads all data from a specified date onwards where depreciated data stopped
    mpx_latest_cases = new_cases(mpx_data, dt.date(2022, 9, 22))
    # Obtains the cases which are not in the rdbs
    mpx_update_case = update_case(mpx_latest_cases)
    # Uploads table for case
    case_df = mpx_update_case.copy()
    for row in range(0,case_df.shape[0]):
        con1 = False
        con2 = False
        in_tuple = tuple(case_df.iloc[row, :])
        if type(in_tuple[3]) != str:
            con1 = True
        if type(in_tuple[5]) != str:
            con2 = True
        if con1 == True and con2 == True:
            case_handler((in_tuple[0], in_tuple[1], in_tuple[2], None, in_tuple[4], None))
        elif con1 == True and con2 == False:
            case_handler((in_tuple[0], in_tuple[1], in_tuple[2], None, in_tuple[4], in_tuple[5]))
        elif con1 == False and con2 == True:
            case_handler((in_tuple[0], in_tuple[1], in_tuple[2], in_tuple[3], in_tuple[4], None))
        else: 
            case_handler(in_tuple)